In [136]:
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [137]:
train_data = pd.read_json('data/train.json')
test_data = pd.read_json('data/test.json')
train_data.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [138]:
train_data.columns

Index([u'bathrooms', u'bedrooms', u'building_id', u'created', u'description',
       u'display_address', u'features', u'interest_level', u'latitude',
       u'listing_id', u'longitude', u'manager_id', u'photos', u'price',
       u'street_address'],
      dtype='object')

In [139]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 10 to 99994
Data columns (total 15 columns):
bathrooms          49352 non-null float64
bedrooms           49352 non-null int64
building_id        49352 non-null object
created            49352 non-null object
description        49352 non-null object
display_address    49352 non-null object
features           49352 non-null object
interest_level     49352 non-null object
latitude           49352 non-null float64
listing_id         49352 non-null int64
longitude          49352 non-null float64
manager_id         49352 non-null object
photos             49352 non-null object
price              49352 non-null int64
street_address     49352 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 6.0+ MB


In [140]:
train_data = train_data.reset_index(drop=True)

In [141]:
interest_dict = {'low':0, 'medium':1, 'high':2}
train_data.replace(to_replace=interest_dict, inplace=True)

In [142]:
#not know how to use the datetime feature
train_data['created'] = pd.to_datetime(train_data['created'], infer_datetime_format=True)

In [102]:
# features = train_data['features'].copy()
# num_features = features.size
# print num_features
# count = 0
# for row in features:
#     if len(row):
#         for f in row:
#             if f not in features_set:
#                 features_set.add(f)
#     count += 1
#     if ((count+1)%1000 == 0):
#         print "Review %d of %d\n" % (count+1, num_features)

In [143]:
stops = set(stopwords.words("english"))
ps = PorterStemmer()
wnl = WordNetLemmatizer()

In [144]:
def text_to_feature(raw_text):
    text = BeautifulSoup(raw_text).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops] 
    clean_words = [wnl.lemmatize(ps.stem(w)) for w in meaningful_words] 
    return( " ".join(clean_words)) 

In [145]:
def fes_to_fes(features):
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops] 
    clean_words = [wnl.lemmatize(ps.stem(w)) for w in meaningful_words] 
    return( " ".join(clean_words)) 

In [146]:
num_des = train_data['description'].size

In [147]:
print "Cleaning and parsing the training set descriptions "
clean_train_descriptions = []
for i in xrange(num_des):
    if ((i+1)%1000 == 0):
        print "Review %d of %d\n" % (i+1, num_des)
    clean_train_descriptions.append(text_to_feature(train_data["description"][i]))

Cleaning and parsing the training set descriptions 
Review 1000 of 49352

Review 2000 of 49352

Review 3000 of 49352

Review 4000 of 49352

Review 5000 of 49352

Review 6000 of 49352

Review 7000 of 49352

Review 8000 of 49352

Review 9000 of 49352

Review 10000 of 49352

Review 11000 of 49352

Review 12000 of 49352

Review 13000 of 49352

Review 14000 of 49352

Review 15000 of 49352

Review 16000 of 49352

Review 17000 of 49352

Review 18000 of 49352

Review 19000 of 49352

Review 20000 of 49352

Review 21000 of 49352

Review 22000 of 49352

Review 23000 of 49352

Review 24000 of 49352

Review 25000 of 49352

Review 26000 of 49352

Review 27000 of 49352

Review 28000 of 49352

Review 29000 of 49352

Review 30000 of 49352

Review 31000 of 49352

Review 32000 of 49352

Review 33000 of 49352

Review 34000 of 49352

Review 35000 of 49352

Review 36000 of 49352

Review 37000 of 49352

Review 38000 of 49352

Review 39000 of 49352

Review 40000 of 49352

Review 41000 of 49352

Review 42000 o

In [148]:
clean_train_descriptions[0]

u'brand new bedroom bath apartmentenjoy follow apart featur rent modern design bathroom w deep spa soak tub room room ac heat real oak hardwood floor rain forest shower head s steel applianc w chef ga cook oven lg fridg washer dryer apt cabl internet readi granit counter top kitchen w lot cabinet storag spaceit block l traindon miss sever great apart immedi area addit inform'

In [149]:
vectorizer = CountVectorizer(analyzer = "word", \
                            tokenizer = None, \
                            preprocessor = None, \
                            stop_words = None, \
                            max_features = 1000)
train_data_features = vectorizer.fit_transform(clean_train_descriptions).toarray()
print train_data_features.shape

word_list = list(vectorizer.get_feature_names())
train_data[word_list] = pd.DataFrame(train_data_features) 

(49352, 1000)


In [150]:
interest_cols = ['bathrooms', 'bedrooms', 'days since created', 'price']

In [151]:
train_data['days since created'] = dt.datetime.now().date() - train_data['created']
train_data['days since created'] = train_data['days since created'].astype('timedelta64[D]')

In [152]:
train_data['features'] = train_data['features'].apply(" ".join)
print "Cleaning and parsing the training set features"
clean_train_features = []
for i in xrange(num_des):
    if ((i+1)%1000 == 0):
        print "Review %d of %d\n" % (i+1, num_des)
    clean_train_features.append(text_to_feature(train_data["features"][i]))

Cleaning and parsing the training set features
Review 1000 of 49352

Review 2000 of 49352

Review 3000 of 49352

Review 4000 of 49352

Review 5000 of 49352

Review 6000 of 49352

Review 7000 of 49352

Review 8000 of 49352

Review 9000 of 49352

Review 10000 of 49352

Review 11000 of 49352

Review 12000 of 49352

Review 13000 of 49352

Review 14000 of 49352

Review 15000 of 49352

Review 16000 of 49352

Review 17000 of 49352

Review 18000 of 49352

Review 19000 of 49352

Review 20000 of 49352

Review 21000 of 49352

Review 22000 of 49352

Review 23000 of 49352

Review 24000 of 49352

Review 25000 of 49352

Review 26000 of 49352

Review 27000 of 49352

Review 28000 of 49352

Review 29000 of 49352

Review 30000 of 49352

Review 31000 of 49352

Review 32000 of 49352

Review 33000 of 49352

Review 34000 of 49352

Review 35000 of 49352

Review 36000 of 49352

Review 37000 of 49352

Review 38000 of 49352

Review 39000 of 49352

Review 40000 of 49352

Review 41000 of 49352

Review 42000 of 493

In [153]:
vectorizer = CountVectorizer(analyzer = "word", \
                            tokenizer = None, \
                            preprocessor = None, \
                            stop_words = None, \
                            max_features = 500)

train_data_fes = vectorizer.fit_transform(clean_train_features).toarray()
train_data_fes.shape

(49352, 500)

In [154]:
f_list = list(vectorizer.get_feature_names())
train_data[f_list] = pd.DataFrame(train_data_fes)  

In [155]:
interest_cols.extend(word_list)
interest_cols.extend(f_list)
len(interest_cols)

1504

In [166]:
new_data = pd.read_json('data/train.json')
new_data = new_data.reset_index(drop=True)

In [167]:
train_data['price'] = new_data['price'].copy()


In [170]:
clean_data = train_data[interest_cols]

In [171]:
clean_data.head()

,bathrooms,bedrooms,days since created,price,aaron,absolut,abund,ac,accent,accept,...,wifi,window,wine,winner,wonder,wood,work,yard,yoga,york
0,1.5,3,379.0,3000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,391.0,5465,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,447.0,2850,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,446.0,3275,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,436.0,3350,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
